In [1]:
import os
import subprocess
import pandas as pd #pip install pandas
import xlwt #pip install xlwt
from openpyxl import load_workbook #pip install openpyxldd

In [2]:
def get_info_row(row):
    result = [] 
    IOD = {"Серийный номер принтера": "1.3.6.1.2.1.43.5.1.1.17.1",
            "Цвет тонера": "1.3.6.1.2.1.43.12.1.1.4.1.1",
            "Максимальное число копий тонера": "1.3.6.1.2.1.43.11.1.1.8.1.1",
            "Оставшееся число копий тонера": "1.3.6.1.2.1.43.11.1.1.9.1.1",
            "Кол-во напечатанных страниц": "1.3.6.1.2.1.43.10.2.1.4.1.1"
            }
    for key, value in IOD.items():
        res = subprocess.check_output(snmp_dir + "SnmpGet.exe -r:" + str(row[2]) + 
                                      ' -o:' + value).decode("cp866").split('\n')
        result.append(res[-2].split('=')[-1])
    return result

In [3]:
snmp_dir = "C:/Users/Никита Широкопетлев/Documents/SnmpGet/"

In [ ]:
wb = load_workbook('./printers.xlsx')
work_sheet = wb.active # Get active sheet

In [ ]:
start_row = 2
cur_row = start_row
for row in work_sheet.iter_rows(min_row=start_row, values_only=True): 
    #print(row[2])
    result = get_info_row(row)
    for col in range(5, 10):
        work_sheet.cell(row=cur_row, column=col).value = result[col - 5].strip()
    cur_row += 1

In [ ]:
wb.save('./printers.xlsx')